In [7]:
import archive
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from termcolor import colored
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [8]:
pio.templates.default = "simple_white"
antq_blue = "#336699"
antq_purple = "#ac39ac"
antq_red = "#e63900"
antq_orange = "#ff8c1a"
faded_grey = "#e6e6e6"
light_grey = "#d9d9d9"
heading_font = "Brush Script MT, Brush Script Std, cursive"
normal_font = "times new roman"
palette = px.colors.qualitative.Set2
palette_r = px.colors.qualitative.Set2_r
set1 = px.colors.qualitative.Set1
set3 = px.colors.qualitative.Set3
warnings.filterwarnings('ignore')

In [9]:
dataset = pd.read_csv(r"Data\Anime.csv")
dataset.head(1000)

,Rank,Name,Japanese_name,Type,Episodes,Studio,Release_season,Tags,Rating,Release_year,End_year,Description,Content_Warning,Related_Mange,Related_anime,Voice_actors,staff
0,1,Demon Slayer: Kimetsu no Yaiba - Entertainment...,Kimetsu no Yaiba: Yuukaku-hen,TV,NaN,ufotable,Fall,"Action, Adventure, Fantasy, Shounen, Demons, H...",4.60,2021.0,NaN,'Tanjiro and his friends accompany the Hashira...,Explicit Violence,Demon Slayer: Kimetsu no Yaiba,"Demon Slayer: Kimetsu no Yaiba, Demon Slayer: ...","Inosuke Hashibira : Yoshitsugu Matsuoka, Nezuk...","Koyoharu Gotouge : Original Creator, Haruo Sot..."
1,2,Fruits Basket the Final Season,Fruits Basket the Final,TV,13.0,TMS Entertainment,Spring,"Drama, Fantasy, Romance, Shoujo, Animal Transf...",4.60,2021.0,NaN,'The final arc of Fruits Basket.',"Emotional Abuse,, Mature Themes,, Physical Abu...","Fruits Basket, Fruits Basket Another","Fruits Basket 1st Season, Fruits Basket 2nd Se...","Akito Sohma : Maaya Sakamoto, Kyo Sohma : Yuum...","Natsuki Takaya : Original Creator, Yoshihide I..."
2,3,Mo Dao Zu Shi 3,The Founder of Diabolism 3,Web,12.0,B.C MAY PICTURES,NaN,"Fantasy, Ancient China, Chinese Animation, Cul...",4.58,2021.0,NaN,'The third season of Mo Dao Zu Shi.',NaN,Grandmaster of Demonic Cultivation: Mo Dao Zu ...,"Mo Dao Zu Shi 2, Mo Dao Zu Shi Q","Lan Wangji, Wei Wuxian, Jiang Cheng, Jin Guang...","Mo Xiang Tong Xiu : Original Creator, Xiong Ke..."
3,4,Fullmetal Alchemist: Brotherhood,Hagane no Renkinjutsushi: Full Metal Alchemist,TV,64.0,Bones,Spring,"Action, Adventure, Drama, Fantasy, Mystery, Sh...",4.58,2009.0,2010.0,"""The foundation of alchemy is based on the law...","Animal Abuse,, Mature Themes,, Violence,, Dome...","Fullmetal Alchemist, Fullmetal Alchemist (Ligh...","Fullmetal Alchemist: Brotherhood Specials, Ful...","Alphonse Elric : Rie Kugimiya, Edward Elric : ...","Hiromu Arakawa : Original Creator, Yasuhiro Ir..."
4,5,Attack on Titan 3rd Season: Part II,Shingeki no Kyojin Season 3: Part II,TV,10.0,WIT Studio,Spring,"Action, Fantasy, Horror, Shounen, Dark Fantasy...",4.57,2019.0,NaN,'The battle to retake Wall Maria begins now! W...,"Cannibalism,, Explicit Violence","Attack on Titan, Attack on Titan: End of the W...","Attack on Titan, Attack on Titan 2nd Season, A...","Armin Arlelt : Marina Inoue, Eren Jaeger : Yuu...","Hajime Isayama : Original Creator, Tetsurou Ar..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Future Diary,Mirai Nikki,TV,26.0,asread,Fall,"Action, Shounen, Battle Royale, High Stakes Ga...",3.97,2011.0,2012.0,"""Yuki is\xa0a disaffected middle school boy wh...","Explicit Violence,, Mature Themes,, Nudity,, S...","Future Diary, Future Diary: Mosaic, Future Dia...","Future Diary OVA, Future Diary: Ura Mirai Nikk...","Yukiteru Amano : Misuzu Togashi, Yuno Gasai : ...","Sakae Esuno : Original Creator, Naoto Hosoda :..."
996,997,Super Lovers OVA,NaN,OVA,2.0,Studio DEEN,NaN,"BL, Romance, Shounen-ai, Based on a Manga",3.97,2017.0,NaN,'No synopsis yet - check back soon!',NaN,Super Lovers,"Super Lovers, Super Lovers 2","Haru Kaidou : Tomoaki Maeno, Ren Kaidou : Junk...",Miyuki Abe : Original Creator
997,998,A Lull in the Sea,Nagi no Asu kara,TV,26.0,P.A. WORKS,Fall,"Drama, Fantasy, Romance, Coming of Age, Love T...",3.97,2013.0,2014.0,"'Long ago, humans lived in the sea. However, s...",NaN,"Nagiyon: Nagi no Asukara 4-koma Gekijou, A Lul...",Ray: Nagi,"Chisaki Hiradaira : Ai Kayano, Hikari Sakishim...","Toshiya Shinohara : Director, Yoshiaki Dewa : ..."
998,999,Future Boy Conan,Mirai Shounen Conan,TV,26.0,Nippon Animation,NaN,"Adventure, Drama, Sci Fi, Post-apocalyptic, Ba...",3.97,1978.0,NaN,"""In AD 2008, mankind developed deadly weapons ...","Mature Themes,, Physical Abuse",NaN,"Mirai Shounen Conan Movie, Mirai Shounen Conan...","Conan : Noriko Ohara, Lana : Mieko Nobusawa, B...",Hayao Miyazaki : Director & Character Design &...
